# HDF-N master catalogue
## Preparation of HST CANDELS-3D data

The catalogue comes from `dmu0_CANDELS-3D-HST`. We no longer use this catalogue since it is superseeded by CANDELS-HDFN

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The kron magnitude, there doesn't appear to be aperture magnitudes. This may mean the survey is unusable.



from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, flux_to_mag

OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "candels_ra"
DEC_COL = "candels_dec"

## I - Column selection

imported_columns = OrderedDict({
        'ID': "candels_id",
        'RAJ2000': "candels_ra",
        'DEJ2000': "candels_dec",
        'S/G':  "candels_stellarity",
        'F140Wap': "f_ap_candels_f140w",
        'e_F140Wap': "ferr_ap_candels_f140w",
        'F140W': "f_candels_f140w",
        'e_F140W': "ferr_candels_f140w",
        'F160Wap': "f_ap_candels_f160w",
        'e_F160Wap': "ferr_ap_candels_f160w",
        'F160W': "f_candels_f160w",
        'e_F160W': "ferr_candels_f160w",
        'F606W': "f_candels_f606w",
        'e_F606W': "ferr_candels_f606w",
        'F814W': "f_candels_f814w",
        'e_F814W': "ferr_candels_f814w",
        'F125W': "f_candels_f125w",
        'e_F125W': "ferr_candels_f125w"

    })


catalogue = Table.read("../../dmu0/dmu0_CANDELS-3D-HST/data/CANDELS-3D-HST_HDF-N.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2012 #Year of publication

#Clean table metadata
catalogue.meta = None

#Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('f_'):
        
        errcol = "ferr{}".format(col[1:])
        
        #Calculate mags, errors including the fact that fluxes are in units of 0.3631 uJy
        mag, error = flux_to_mag(np.array(catalogue[col]) * 0.3631e-6, np.array(catalogue[errcol] * 0.3631e-6))
        
        
        # magnitudes are added
        catalogue.add_column(Column(mag, name="m{}".format(col[1:])))
        catalogue.add_column(Column(error, name="m{}".format(errcol[1:])))
        
        
        #Correct flux units to uJy
        catalogue[col] = catalogue[col] * 0.3631
        catalogue[col].unit = u.microjansky
        catalogue[errcol] = catalogue[errcol] * 0.3631
        catalogue[errcol].unit = u.microjansky        

        if ('125' in col) or ('814' in col) or ('606' in col) :
            # We add nan filled aperture photometry for consistency
            catalogue.add_column(Column(np.full(len(catalogue), np.nan), name="m_ap{}".format(col[1:])))
            catalogue.add_column(Column(np.full(len(catalogue), np.nan), name="merr_ap{}".format(col[1:])))
            catalogue.add_column(Column(np.full(len(catalogue), np.nan), name="f_ap{}".format(col[1:])))
            catalogue.add_column(Column(np.full(len(catalogue), np.nan), name="ferr_ap{}".format(col[1:])))

        

        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
#TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.


SORT_COLS = ['merr_candels_f140w', 
             'merr_candels_f160w', 
             'merr_candels_f606w', 
             'merr_candels_f814w',
             'merr_candels_f125w']
FLAG_NAME = 'candels_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_HDF-N.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

GAIA_FLAG_NAME = "candels_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

catalogue.write("{}/CANDELS.fits".format(OUT_DIR), overwrite=True)